# **HMMs**

Group 5: Xueying He, Kawthar Shafie Khorassani, Jinhee Lee , Jared Walker

# Extension 0

**Mandatory part (aka Part 0, 15 points):  Implement a HMM bigram tagger using the Viterbi algorithm**



*   Read the word/tag pairs from pos_train.txt (example in create_counts.py) and count word given tag and tag given previous tag.
*   Use these counts to compute probability distributions.  Note that you will probably want to make probabilities in the log domain to prevent underflow.
*   Implement the Viterbi algorithm using bi-tag (bigram) statistics (the probability of a tag given the previous tag) as well as probability of word given tag
*   Using the sentences in pos_test.txt, compute the most likely tag for the words in each sentence.  (Assume unknown words have some small probably of belonging to the most frequent tag.) Compare against the tags in the test set.  Report the word-level tagging accuracy.

In [ ]:
from google.colab import files
data=files.upload()    # upload pos_train.txt into Colab 
test=files.upload()    # upload pos_test.txt into Colab  

Saving pos_train.txt to pos_train (2).txt


Saving pos_test.txt to pos_test (1).txt


In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

# this is an example of how to parse the POS tag file and get counts
# needed for a bigram tagger

tag_given_tag_counts = dict()  # count of each tag
word_given_tag_counts = dict() # count 

with open("pos_train.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file 
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
            tag = parts.pop()   # only shows tag of this word  ex) NNS           
            word = "/".join(parts)   # only shows a word   ex) years

            # update counters
            if tag not in word_given_tag_counts:          # add new tag in word dictionary        
                word_given_tag_counts[tag] = Counter()                
            if lasttag not in tag_given_tag_counts:       # add new tag in tag dictionary
                tag_given_tag_counts[lasttag] = Counter()
            word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
            tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
            lasttag = tag          
       


# examples
print("count[NN][VB] = " + str(tag_given_tag_counts["NN"]["VB"]))   # means  after tag 'NN' , 'VB' shows 229 times.   t1 is 'NN' t2 is 'VB' and  result is Count(t1, t2)
print("count[NN][dog] = " + str(word_given_tag_counts["NN"]["dog"]))  # means  after tag 'NN', 'dog' shows 12 times.  t1 is 'NN' w1 is 'dog' and  result is Count(t1, w1)


count[NN][VB] = 229
count[NN][dog] = 12


In [ ]:
# Make Dataframe of each tag_tag and tag_word

def getDataframe(tag_given_tag_counts, word_given_tag_counts):
  # row is ti-1 and col is ti
  tagData=pd.DataFrame(tag_given_tag_counts)
  tagData=tagData.fillna(0)
  tagData=tagData.transpose()
  #print(tagData)

  wordData=word_given_tag_counts
  wordData=pd.DataFrame(wordData)
  wordData=wordData.fillna(0)
  wordData=wordData.transpose()
  #print(wordData)


  #print(list(wordData.columns))

  #print(tagData.loc["NNP"].tolist())
  #print(wordData.loc["NNP"].tolist())
  #print(word_given_tag_counts["NNP"]["rarsasrtat"])

  return tagData, wordData

#print(wordData.columns.tolist())
#print(str(word_given_tag_counts["NNP"]))

#tagData, wordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)

#print(tagData)
#print(wordData)

In [ ]:
# we have to calcualte P(ti | ti_1) = count(ti-1, ti)/count(ti-1)

def getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts):
  # get P(ti|ti-1)
  probtag=tagData.copy()  # to make probability dataframe

  for ti_1 in tag_given_tag_counts.keys():  # get ti-1
    count_ti_1=tagData.loc[ti_1]    # Count of tags in this ti_1 tag
    sumtag=int(np.sum(count_ti_1))   # C(ti-1)
    
    for ti in tag_given_tag_counts.keys():  # get ti 
      if ti != "<s>":      
        tagcount=int(tagData.loc[ti_1][ti])   # this is count(ti-1, ti):   count(ti) in tag ti_1           

        prob_ti_ti_1=tagcount/sumtag   # P(ti | ti-1)
        if prob_ti_ti_1 == 0:
          probtag.loc[ti_1][ti] = 0.000001   # if 0 probability: count 0 words, make it as very small probability
        else:
          probtag.loc[ti_1][ti]=prob_ti_ti_1

  #print(probtag)   # each element is P(ti|ti-1)  ex) P(NNP|<s>) = 0.282620
    

  # get P(wi|ti)
  probword=wordData.copy()
  words=wordData.columns.tolist()  # all words

  for ti in word_given_tag_counts.keys():  # get ti (tag)
    count_ti=tagData.loc[ti]
    sumtag=int(np.sum(count_ti))

    for wi in word_given_tag_counts[ti]:  # get wi (word)         
      wordcount=int(wordData.loc[ti][wi])

      prob_wi_ti=wordcount/sumtag   # P(wi | ti)
      probword.loc[ti][wi]=prob_wi_ti

  probword+=0.000001  # to erase zeros in probability distribution,  add very small probability

  #print(probword)  # each element is P(wi|ti)  ex) P(Pierre|NNP) = 0.000052
  #print(probword.loc["NNP"]["Pierre"])
  #print(len(word_given_tag_counts["NNP"]))

  return probtag, probword


#probtag, probword=getProb(tagData, wordData)
#print(probtag)
#print(probword)

In [ ]:
# get log P(ti|ti-1)
def logprob(probtag, probword):
  logprobtag=probtag.copy()  # to get log probability distribution of tag_tag pair

  logprobtag=np.log(logprobtag)

  #print(logprobtag)   # each element is log P(ti|ti-1)  ex) log P(NNP|<s>) = -1.262651
    

  # get log P(wi|ti)
  logprobword=probword.copy()   # to get log probability distribution of word_tag pair
  logprobword=np.log(logprobword)

  #print(logprobword)  # each element is log P(wi|ti)  ex) log P(Pierre|NNP) = -9.838250
  return logprobtag, logprobword

#logprobtag, logprobword=logprob(probtag, probword)
#print(logprobtag)
#print(logprobword)

In [ ]:
# just check the Viterbi Algorithm, take one sentence in the dataset

sentences=[]
with open("pos_train.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        sentence={}
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']
            #print(parts[1], parts[0])            
            sentence[parts[0]]=parts[1]
            #tag = parts.pop()   # only shows tag of this word  ex) NNS           
            #word = "/".join(parts)   # only shows a word   ex) years            
            #sentence[tag]=word        
        sentences.append(sentence)
        
        
# one sentence        
sent=sentences[10]
print(sent)

{'The': 'DT', 'percentage': 'NN', 'of': 'IN', 'lung': 'NN', 'cancer': 'NN', 'deaths': 'NNS', 'among': 'IN', 'the': 'DT', 'workers': 'NNS', 'at': 'IN', 'West': 'NNP', 'Groton': 'NNP', ',': ',', 'Mass.': 'NNP', 'paper': 'NN', 'factory': 'NN', 'appears': 'VBZ', 'to': 'TO', 'be': 'VB', 'highest': 'JJS', 'for': 'IN', 'any': 'DT', 'asbestos': 'NN', 'studied': 'VBN', 'in': 'IN', 'Western': 'JJ', 'industrialized': 'VBN', 'countries': 'NNS', 'he': 'PRP', 'said': 'VBD', '.': '.', 'plant': 'NN', 'which': 'WDT', 'is': 'VBZ', 'owned': 'VBN', 'by': 'IN', 'Hollingsworth': 'NNP', '&': 'CC', 'Vose': 'NNP', 'Co.': 'NNP', 'was': 'VBD', 'under': 'IN', 'contract': 'NN', 'with': 'IN', 'Lorillard': 'NN', 'make': 'VB', 'cigarette': 'NN', 'filters': 'NNS'}


In [ ]:
# Viterbi Algorithm

#tags=list(sent.keys())
#words=list(sent.values())

tagData,wordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)
probtag, probword=getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts)
logprobtag, logprobword=logprob(probtag, probword)


## use log probability
def viterbi(sentence, logprobtag, logprobword):  # sequence is a sentence
  #print("hidden: ", list(sentence.keys()))
  #print("observation: ", list(sentence.values()))

  
  tags=logprobtag.columns    # tags of the transition Matrix except <s>
  words=list(sentence.keys())   # words in this sentence
  
 
  # Initialization
  # M = P(tagi) * P(wordi|tagi) as viterbi probability : prior * emission
  M={}  # at start, P(tagi) = P(tagi| "<s>" )
  for tag in tags:     # tags of this sentenc words
    M[tag]=logprobtag.loc["<s>"][tag] + logprobword.loc[tag][words[0]]  # word[0] is the first word of this sentence

  pre=[]

  # Vt(j) = max( Vt-1(i) * P(tagj|tagi) * P(wordj|tagj) )
  
  for t in range(1, len(sentence)):   # Current state: t        
 
    probVit={}   # viterbi probability at t State:  P(Viterbi at j State) * P(tagj|tagi) * P(wordj|tagj)
                # this will store previous viterbi probabilities of current State

    preState={}  # this will store previous tags of current State

    for j in tags:     # j  Current State
      max_prob=-10000  # just set very small number  prob=0 -> logprob ~ very small number
      max_state=None

      for i in tags:   # i   previous State 
                                            #      previous Viterbi
        prob=M[i] + logprobtag.loc[j][i]  #  P(Viterbi at i State) * P(tagj|tagi)
        #print(prev[tagii], logprobtag.loc[tagi][tagj], '   ', prob) 

        #print(prob, max_prob) 
              
        if prob > max_prob:  # change max probability     
          max_prob=prob      # get max of P(Viterbi at i State) * P(tagj|tagi)
          max_state=i  # tag which has a maximum probability 

      if words[t] not in logprobword.columns:  # if this word is not in the dictionary, set prob as very small number
        probV=max_prob - 100
      else:
        probV = max_prob + logprobword.loc[j][words[t]]  # viterbi probability of this State:   P(Viterbi at i State) * P(tagj|tagi) * P(wordi|tagi)

      probVit[j] = probV     # store viterbi probability of this State into this dictionary
      preState[j]=max_state  # { 'DT' : 'IN' } means at tag 'DT' of this sentence, 'IN' shows maximum probability  
    M=probVit
    pre.append(preState)

  #print(M)
  # find hidden states with back tracking
  max_state=None     
  max_prob=-10000
  for tag in tags:    
    if M[tag] > max_prob:
      max_prob=M[tag]
      max_state=tag

  result = [max_state]   # last word's tag in this sentence
  #print(result)

  for t in range(len(sentence)-1, 0, -1):  # from last -> first word 
    max_state=pre[t-1][max_state]

    result.insert(0, max_state)

  #print("Predicted tags: ", result)

  return result
    

trainVit=viterbi(sent,logprobtag, logprobword)
print(trainVit)
  
  



['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'DT', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'DT', 'TO', 'VB', 'JJS', 'IN', 'DT', 'IN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'TO', 'NNS']


In [ ]:
# Get train data's accuracy.

print("hidden: ", list(sent.values()))
print("Predict:", trainVit)

trainTags=list(sent.values())

total=len(trainVit)
correct=0
for i in range(len(trainVit)):
  if trainVit[i] == trainTags[i]:
    correct+=1

print("Accuracy: ",correct/total)


hidden:  ['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'DT', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'VBZ', 'TO', 'VB', 'JJS', 'IN', 'DT', 'NN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'NN', 'NNS']
Predict: ['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'DT', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'DT', 'TO', 'VB', 'JJS', 'IN', 'DT', 'IN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'TO', 'NNS']
Accuracy:  0.9375


In [ ]:
# Apply my Viterbi algorithm to the test dataset

# Get test.txt's tags and words
test_tag_given_tag_counts = dict()  # count of each tag
test_word_given_tag_counts = dict() # count 

with open("pos_test.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file 
               
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
            tag = parts.pop()   # only shows tag of this word  ex) NNS           
            word = "/".join(parts)   # only shows a word   ex) years

            # update counters
            if tag not in test_word_given_tag_counts:          # add new tag in word dictionary        
                test_word_given_tag_counts[tag] = Counter()                
            if lasttag not in test_tag_given_tag_counts:       # add new tag in tag dictionary
                test_tag_given_tag_counts[lasttag] = Counter()
            test_word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
            test_tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
            lasttag = tag          

testtagData,testwordData=getDataframe(test_tag_given_tag_counts, test_word_given_tag_counts)
testprobtag, testprobword=getProb(testtagData, testwordData,test_tag_given_tag_counts, test_word_given_tag_counts)
testlogprobtag, testlogprobword=logprob(testprobtag, testprobword)

In [ ]:
test_sentences=[]
with open("pos_test.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        test_sentence={}
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']
            #print(parts[1], parts[0])            
            test_sentence[parts[0]]=parts[1]
            #tag = parts.pop()   # only shows tag of this word  ex) NNS           
            #word = "/".join(parts)   # only shows a word   ex) years            
            #sentence[tag]=word        
        test_sentences.append(test_sentence)
        
        
# one sentence        
test_sent=test_sentences[6]
print(test_sent)

{'Countered': 'VBD', 'James': 'NNP', 'Maguire': 'NNP', ',': ',', 'chairman': 'NN', 'of': 'IN', 'specialists': 'NNS', 'Henderson': 'NNP', 'Brothers': 'NNP', 'Inc.': 'NNP', ':': ':', '``': '``', 'It': 'PRP', 'is': 'VBZ', 'easy': 'JJ', 'to': 'TO', 'say': 'VB', 'the': 'DT', 'specialist': 'NN', "n't": 'RB', 'doing': 'VBG', 'his': 'PRP$', 'job': 'NN', '.': '.', 'When': 'WRB', 'dollar': 'NN', 'in': 'IN', 'a': 'DT', 'free-fall': 'NN', 'even': 'RB', 'central': 'JJ', 'banks': 'NNS', 'ca': 'MD', 'stop': 'VB', 'it': 'PRP', 'Speculators': 'NNS', 'are': 'VBP', 'calling': 'VBG', 'for': 'IN', 'degree': 'NN', 'liquidity': 'NN', 'that': 'WDT', 'not': 'RB', 'there': 'RB', 'market': 'NN', "''": "''"}


In [ ]:
# test Viterbi algorithm for just one sentence in the test dataset
testVit=viterbi(test_sent, testlogprobtag, testlogprobword)
print(testVit)

['NNP', 'NNP', 'NNP', ',', 'NN', 'IN', 'NNS', 'NNP', 'NNP', 'NNP', ':', '``', 'PRP', 'VBZ', 'JJ', 'TO', 'VBP', 'DT', 'IN', 'RB', 'VBG', 'PRP$', 'NN', '.', 'WRB', 'NN', 'IN', 'DT', 'NN', 'JJ', 'JJ', 'NNS', 'JJ', 'VB', 'PRP', 'NNS', 'VBP', 'NNS', 'IN', 'NN', 'NN', 'IN', 'RB', 'RB', 'NN', "''"]


In [ ]:
# Report test Accuracy of each sentence
# It takes too much time to get all result of the entire test dataset.
Accuracy=[]
count=0
for i in test_sentences:

  if count==30:
    break
  
    
  for j in i.keys():
    print(j, end=" ")
  print()

  v=viterbi(i,testlogprobtag, testlogprobword)

  #print("hidden State: ", list(i.keys()))
  #print("Predicted State :", v)

  b=list(i.values())

  total=len(v)
  correct=0
  for j in range(len(v)):
    if v[j] == b[j]:
      correct+=1

  acc=correct/total
  Accuracy.append(acc)
  print("Accuracy: ", acc)
  count+=1

No , it was n't Black Monday . 
Accuracy:  0.875
But while the New York Stock Exchange did n't fall apart Friday as Dow Jones Industrial Average plunged 190.58 points -- most of it in final hour barely managed to stay this side chaos . 
Accuracy:  0.8571428571428571
Some `` circuit breakers '' installed after the October 1987 crash failed their first test , traders say unable to cool selling panic in both stocks and futures . The 49 stock specialist firms on Big Board floor -- buyers sellers of last resort who were criticized once again could n't handle pressure 
Accuracy:  0.8679245283018868
Big investment banks refused to step up the plate support beleaguered floor traders by buying big blocks of stock , say . Heavy selling Standard & Poor 's 500-stock index futures in Chicago relentlessly beat stocks downward 
Accuracy:  0.7297297297297297
Seven Big Board stocks -- UAL , AMR BankAmerica Walt Disney Capital Cities\ Philip Morris and Pacific Telesis Group stopped trading never resumed

In [ ]:
# Viterbi Algorithm's accuracy of test dataset 
print(Accuracy)
mean_Accuracy=np.mean(Accuracy)
print(mean_Accuracy)

[0.875, 0.8571428571428571, 0.8679245283018868, 0.7297297297297297, 0.9166666666666666, 0.9047619047619048, 0.8478260869565217, 0.88, 0.8837209302325582, 0.8867924528301887, 0.8297872340425532, 0.8541666666666666, 0.8181818181818182, 0.8636363636363636, 0.8775510204081632, 0.8367346938775511, 0.8285714285714286, 0.8163265306122449, 0.7894736842105263, 0.7872340425531915, 0.95, 0.896551724137931, 0.8571428571428571, 0.8787878787878788, 0.8529411764705882, 0.9310344827586207, 0.8536585365853658, 0.9565217391304348, 0.8163265306122449, 0.7872340425531915]
0.8577142535853977


In [ ]:
# We tried Viterbi for the rntire test dataset.
# We got: mean of Viterbi accuracy for test dataset as 0.8634231305868963 :  86.34%: all test sentences  <- We did it.  However, in here, get mean of only 30 sentences
print("Average Viterbi Accuracy: ", mean_Accuracy)

Average Viterbi Accuracy:  0.8577142535853977


# Extension 1 Attempt

**Extension 1 (10 points)**: Implement the forward-backward algorithm to estimate the probabilities of the tags assuming you did not have a labeled text -- that is:


*   Initialization: Use the counts from pos_train.txt to figure out which tags are possible for words, and which bigram tags are possible, but then ignore the counts (i.e. make all counts equal to 1) to initialize the probabilities.  
*   Expectation: You should use the words from pos_train.txt but not the tags.  Compute the probability of all tags for each word (given the sequence and alpha-beta recursion), as well as the bigram tag probability.
*   Maximization: Use the tag probabilities as soft counts to update your P(word|tag) and P(tag|previous tag).
*   Loop between the E and M steps until you converge (or pick a fixed number of iterations)
*   Now use the Viterbi algorithm to compute the best tags for the test set.  Compare the results to those you obtained from Part 0 (when you knew the tags).  Is it better?  Worse?
*   **HINT!!!**  If you test your code using Eisner's ice cream example, you can check your work against the spreadsheet.  This will VASTLY simplify your debugging.






In [ ]:
# Upload training data file
train_file = files.upload()

Saving pos_train.txt to pos_train (3).txt


In [ ]:
tag_given_tag_counts=dict()
word_given_tag_counts=dict()

word_sequence = []
with open("pos_train.txt", "r") as infile: 
  for line in infile:
      #
      # first tag is the start symbol
      lasttag="<s>"
      #
      # split line into word/tag pairs
      #
      for wordtag in str(line).rstrip().split(" "):
          if wordtag == "":
              continue
          # note that you might have escaped slashes
          # 1\/2/CD means "1/2" "CD"
          # keep 1/2 as 1\/2 
          parts=wordtag.split("/")
          tag=parts.pop()
          word="/".join(parts)

          word_sequence.append(word)

          #If statement
          if tag not in word_given_tag_counts:
              word_given_tag_counts[tag]=Counter()
          if lasttag not in tag_given_tag_counts:
              tag_given_tag_counts[lasttag]=Counter()
          word_given_tag_counts[tag][word]=1
          tag_given_tag_counts[lasttag][tag]=1
          lasttag=tag
# print(word_given_tag_counts)

In [ ]:
## Obtain state sequence
tag_sequence = []
for tag in word_given_tag_counts.keys():
  if tag not in tag_sequence:
    tag_sequence.append(tag)

In [ ]:
tag_given_tag = pd.DataFrame(tag_given_tag_counts)
word_given_tag = pd.DataFrame(word_given_tag_counts)
tag_given_tag = tag_given_tag.fillna(0)
tag_given_tag = tag_given_tag.transpose()

word_given_tag = word_given_tag.fillna(0)
word_given_tag = word_given_tag.transpose()

In [ ]:
print(tag_given_tag)
print(word_given_tag)

          NNP   DT   IN   ``  NNS  ...  VBP|VB  VBD|VBP  VBP|VBD  JJ|JJR  IN|JJ
<s>       1.0  1.0  1.0  1.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
NNP       1.0  1.0  1.0  1.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
,         1.0  1.0  1.0  1.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
CD        1.0  1.0  1.0  1.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
NNS       1.0  1.0  1.0  1.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
...       ...  ...  ...  ...  ...  ...     ...      ...      ...     ...    ...
NN|POS    0.0  0.0  0.0  0.0  0.0  ...     0.0      0.0      0.0     0.0    0.0
IN|JJ     0.0  0.0  0.0  0.0  1.0  ...     0.0      0.0      0.0     0.0    0.0
JJ|VBN    0.0  0.0  0.0  0.0  0.0  ...     0.0      0.0      0.0     0.0    0.0
NNS|NNPS  0.0  0.0  0.0  0.0  0.0  ...     0.0      0.0      0.0     0.0    0.0
VBD|VBN   0.0  0.0  0.0  0.0  0.0  ...     0.0      0.0      0.0     0.0    0.0

[80 rows x 79 columns]
          Pierre

In [ ]:
## Obtain transition probability

# Obtain total counts of each previous tag
last_tag_total_counts = []
count = 0
sum_counts = np.array([])
for i in range(len(tag_given_tag)):
  sum_counts = np.array(tag_given_tag.iloc[i, :])
  count = int(np.sum(sum_counts))
  last_tag_total_counts.append(count)

last_tag_total_counts = np.array(last_tag_total_counts)

# Obtain transition probability
transition_probs = tag_given_tag.copy()
for i in range(len(tag_given_tag)):
  for j in range(len(tag_given_tag.columns)):
    tagCount = int(tag_given_tag.iloc[i, j])
    sum_tagCount = int(last_tag_total_counts[i])
    if tagCount == 0 or sum_tagCount == 0:
      break
    transition_probs.iloc[i, j] = (tagCount / sum_tagCount)

In [ ]:
#Normalize each column in transition_probs
for tag in transition_probs.columns.values:
  trans_prob = np.array(transition_probs.loc[:,tag])
  sum = np.sum(trans_prob)
  for row in transition_probs.index.values:
    norm_trans_probs = transition_probs.loc[row][tag] / sum
    transition_probs.loc[row][tag] = norm_trans_probs

In [ ]:
print(transition_probs)

               NNP        DT        IN  ...  VBP|VBD  JJ|JJR  IN|JJ
<s>       0.009523  0.003552  0.001781  ...      0.0     0.0    0.0
NNP       0.008858  0.003304  0.001657  ...      0.0     0.0    0.0
,         0.009069  0.003383  0.001696  ...      0.0     0.0    0.0
CD        0.009069  0.003383  0.001696  ...      0.0     0.0    0.0
NNS       0.008657  0.003229  0.001619  ...      0.0     0.0    0.0
...            ...       ...       ...  ...      ...     ...    ...
NN|POS    0.000000  0.000000  0.000000  ...      0.0     0.0    0.0
IN|JJ     0.000000  0.000000  0.000000  ...      0.0     0.0    0.0
JJ|VBN    0.000000  0.000000  0.000000  ...      0.0     0.0    0.0
NNS|NNPS  0.000000  0.000000  0.000000  ...      0.0     0.0    0.0
VBD|VBN   0.000000  0.000000  0.000000  ...      0.0     0.0    0.0

[80 rows x 79 columns]


In [ ]:
## Obtain emission probability

# Obtain total counts of each tag
given_word_tag_counts = []
count = 0
sum_counts = np.array([])
for i in range(len(word_given_tag)):
  sum_counts = np.array(tag_given_tag.iloc[i, :])
  count = int(np.sum(sum_counts))
  given_word_tag_counts.append(count)

given_word_tag_counts = np.array(given_word_tag_counts)

# Obtain emission probability
emission_probs = word_given_tag.copy()
for i in range(len(word_given_tag)):
  for j in range(len(word_given_tag.columns)):
    tagCount = int(word_given_tag.iloc[i, j])
    sum_tagCount = int(given_word_tag_counts[i])
    if tagCount == 0 or sum_tagCount == 0:
      break
    emission_probs.iloc[i, j] = (tagCount / sum_tagCount)

columns_index = transition_probs.columns.values
emission_probs = emission_probs.reindex(columns_index)
print(emission_probs)

         Pierre  Vinken   Nov.  ...  zillion  then-Air  less-advanced
NNP       0.025   0.025  0.025  ...      0.0       0.0            0.0
DT        0.000   0.000  0.000  ...      0.0       0.0            0.0
IN        0.000   0.000  0.000  ...      0.0       0.0            0.0
``        0.000   0.000  0.000  ...      0.0       0.0            0.0
NNS       0.000   0.000  0.000  ...      0.0       0.0            0.0
...         ...     ...    ...  ...      ...       ...            ...
VBP|VB    0.000   0.000  0.000  ...      0.0       0.0            0.0
VBD|VBP   0.000   0.000  0.000  ...      0.0       0.0            0.0
VBP|VBD   0.000   0.000  0.000  ...      0.0       0.0            0.0
JJ|JJR    0.000   0.000  0.000  ...      0.0       0.0            1.0
IN|JJ     0.000   0.000  0.000  ...      0.0       0.0            0.0

[79 rows x 49343 columns]


In [ ]:
#Normalize each column in emission_probs
i = 0
for tag in emission_probs.columns.values:
  emiss_prob = np.array(emission_probs.loc[:,tag])
  sum = np.sum(emiss_prob)
  for row in emission_probs.index.values:
    norm_trans_probs = emission_probs.loc[row][tag] / sum
    emission_probs.loc[row][tag] = norm_trans_probs
  i += 1
  # print(i)

In [ ]:
print(emission_probs)

         Pierre  Vinken      Nov.  ...  zillion  then-Air  less-advanced
NNP         1.0     1.0  0.012346  ...      0.0       0.0            0.0
DT          0.0     0.0  0.000000  ...      0.0       0.0            0.0
IN          0.0     0.0  0.000000  ...      0.0       0.0            0.0
``          0.0     0.0  0.000000  ...      0.0       0.0            0.0
NNS         0.0     0.0  0.000000  ...      0.0       0.0            0.0
...         ...     ...       ...  ...      ...       ...            ...
VBP|VB      0.0     0.0  0.000000  ...      0.0       0.0            0.0
VBD|VBP     0.0     0.0  0.000000  ...      0.0       0.0            0.0
VBP|VBD     0.0     0.0  0.000000  ...      0.0       0.0            0.0
JJ|JJR      0.0     0.0  0.000000  ...      0.0       0.0            1.0
IN|JJ       0.0     0.0  0.000000  ...      0.0       0.0            0.0

[79 rows x 49343 columns]


In [ ]:
##Forward Backward Implementation

#While loop over all training data

##Calculate Alpha-Beta Recursion

#Initialize

transition_probs+=0.000001
emission_probs+=0.000001
prior_probs=transition_probs.loc["<s>"]

logtransition_probs=np.log(transition_probs)
logemission_probs=np.log(emission_probs)
logprior_probs=np.log(prior_probs)

def forward(sentence):
  
  tags=list(tag_given_tag.columns)   # tags of the transition Matrix except <s>  
  words=list(sentence.keys())   # words in this sentence
 
  # Initialization
  alpha=[{}]

  for tag in tags:     # tags of this sentenc words    
    alpha[0][tag]= logprior_probs[tag] + logemission_probs.loc[tag][words[0]]  
  #print(alpha)

  for t in range(1, len(sentence)): 
    alpha.append({})    
    for j in tags:
      prob=0
      for i in tags:
        aa=alpha[t-1][i]+ logtransition_probs.loc[j][i]
        prob+= np.exp(aa)
      
      alpha[t][j]= np.log(prob) + logemission_probs.loc[j][words[t]]
     
  return alpha
      
alpha=forward(sent)
print(alpha)

[{'NNP': -9.0468706763603, 'DT': -6.343174795568927, 'IN': -17.943867566785244, '``': -16.669972600068625, 'NNS': -18.004021042844457, 'JJ': -17.693167184929074, 'RB': -17.857224974240125, 'WRB': -15.858488977766147, 'JJR': -17.34223010831025, 'NN': -18.581293775556873, '(': -17.665274690200427, 'CD': -18.04955785820831, 'EX': -17.01285281035595, 'VBG': -18.105572945790396, 'VB': -7.251265425819493, 'PRP': -18.043473416235482, 'CC': -18.105572945790396, 'WP': -17.446062891831094, 'TO': -18.321016754335513, 'VBN': -18.321016754335513, 'PRP$': -17.977211667166138, 'VBD': -18.21916241452257, 'LS': -16.065467837367997, ':': -18.795936048580486, 'VBZ': -18.853830230526263, 'VBP': -18.76564873376374, 'WDT': -18.396446345993517, 'NNPS': -18.522469727929888, 'JJS': -18.56112516976587, 'RBS': -18.396446345993517, 'RBR': -18.522469727929888, 'PDT': -18.35057669097531, 'SYM': -16.93275811874604, 'FW': -18.39440284798085, 'UH': -17.874369130474577, '$': -18.732945044632125, 'MD': -18.7945767221480

In [ ]:
def backward(sentence):
  tags=list(tag_given_tag.columns)   # tags of the transition Matrix except <s>  
  words=list(sentence.keys())   # words in this sentence
 
  beta=[{}]

  for tag in tags:     # tags of this sentenc words    log(1)=0
    beta[0][tag]= 0
  #print(beta)

  for t in range(len(sentence)-1, 0, -1): 
    beta.insert(0, {})    
    for j in tags:     
      prob=0    
      for i in tags:
        aaa= beta[1][j]+ logtransition_probs.loc[j][i]
        bbb= aaa + logemission_probs.loc[i][words[t]]
        prob += np.exp(bbb)

      beta[0][j] = np.log(prob)

  return beta

beta=backward(sent)
print(beta)

[{'NNP': -255.89458948406855, 'DT': -227.83463062884957, 'IN': -308.21200246437894, '``': -284.6559835577367, 'NNS': -256.8383145784233, 'JJ': -263.0206510822041, 'RB': -262.8976777029776, 'WRB': -255.5448690373671, 'JJR': -189.78570296464122, 'NN': -261.2058414409982, '(': -246.10909470756016, 'CD': -223.32495473094136, 'EX': -353.6845295546295, 'VBG': -256.22337169036774, 'VB': -262.6280815940985, 'PRP': -260.753954866718, 'CC': -293.70313062321975, 'WP': -203.6138189226343, 'TO': -238.77601244087262, 'VBN': -259.066172689163, 'PRP$': -167.76054966483417, 'VBD': -261.32702017179577, 'LS': -355.4528013204249, ':': -300.5665478561062, 'VBZ': -262.5513601512656, 'VBP': -261.2942397055398, 'WDT': -205.9956999364534, 'NNPS': -205.19965837998194, 'JJS': -207.0841892137241, 'RBS': -190.13326418853285, 'RBR': -177.4204125513272, 'PDT': -467.95140061387156, 'SYM': -235.15649565396873, 'FW': -228.00361116356433, 'UH': -309.31738656994054, '$': -481.85121181263645, 'MD': -199.45986198726695, 'R

In [ ]:
def expectation(sentence):

   tags=list(tag_given_tag.columns)   # tags of the transition Matrix except <s>  
   words=list(sentence.keys())   # words in this sentence   

   alpha=forward(sentence)
   beta=backward(sentence)
   
   gamma=[]
   for t in range(len(sentence)):
     prob_sum=0
     gamma.append({})
     for tag in tags:       
       prob=alpha[t][tag] + beta[t][tag]       
       gamma[t][tag]=prob
       prob_sum+=prob
          
     if prob_sum==0:
          gamma[t][tag]=-10000 
          continue
     else:
        for tag in tags:          
          gg = gamma[t][tag]/prob_sum
          gamma[t][tag] = np.log(gg)
   print(gamma)
   

   xi=[]
   for t in range(len(sentence)-1):
     prob_sum=0
     xi.append({})
     for j in tags:
       xi[t][j]={}
       for i in tags:
         te= logtransition_probs.loc[j][i] + logemission_probs.loc[i][words[t+1]] 
         prob= alpha[t][j] + beta[t+1][i] + te
         xi[t][j][i]= prob
         prob_sum+=prob
     
     if prob_sum ==0:       
       continue
     else:      
      for j in tags:
          for i in tags:            
            xx = xi[t][j][i]/prob_sum
            xi[t][j][i]= np.log(xx)
   print(xi)
          
   return gamma, xi


gamma, xi =expectation(sent)

In [ ]:
print(gamma)
print(xi)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
def maximization(sentence, smoothing=0):
  tags=list(tag_given_tag.columns)   # tags of the transition Matrix except <s>  
  words=list(sentence.keys())   # words in this sentence

  gamma=np.exp(gamma)
  xi=np.exp(xi)

  for tag in tags:
    prior_probs[tag] = smoothing+gamma[0][tag] / (1+len(tag)*smoothing)
  
    gamma_sum=0
    for t in range(len(sentence)-1):
      gamma_sum +=gamma[t][tag]

    if gamma_sum >0:
      denom=gamma_sum+len(tag)*smoothing
      for i in tags:
        xi_sum=0
        for t in range(len(sentence)-1):
          xi_sum += xi[t][tag][i]
        aa=(smoothing + xi_sum)/denom
        transition_probs.loc[tag][i] = np.log(aa)

    else:
      for i in tags:
        aa=0.0000001
        transition_probs.loc[tag][i] = np.log(aa)
    gamma_sum += gamma[len(sentence)-1][tag]

    emit_gamma_sum={}
    for word in words:
      emit_gamma_sum[word]=0
    for t in range(len(sentence)):
      emit_gamma_sum[words[t]] += gamma[t][tag]

    if gamma_sum >0:
      denom=gamma_sum+len(words)+smoothing
      for word in words:
        bb=(smoothing + emit_gamma_sum[word])/denom
        emission_probs.loc[tag][word] = np.log(bb)
    else:
      for word in words:
        bb=0.000001
        emission_probs.loc[tag][word]=np.log(bb)
  return transition_probs, emission_probs


transition_probs, emission_probs=maximization(sent, smoothing=0)


In [ ]:
# it takes too much time to converge. 
numiter=3
for i in range(numiter):
  print(i)
  gamma, xi =expectation(sent)
  transition_probs, emission_probs=maximization(sent, smoothing=0)


# Extension 2

**Extension 2 (5 points): Viterbi training - this is similar to forward-backward except you make a hard decision at every step.  This should be much simpler.**

* Initialization: Use the counts from pos_train.txt to figure out which tags are possible for words, and which bigram tags are possible, but then ignore the counts (i.e. make all counts equal to 1) to initialize the probabilities.

* Viterbi: You should use the words from pos_train.txt but not the tags.  Use the Viterbi algorithm to find the most likely tag sequence for the training set words.  Write out a file with the word/tag sequences similar to the pos_train.txt file (e.g. pos_train.iter1.txt).

* Estimation: Now read in the file you just wrote out using your count-based estimator to get new probability distributions.

* Loop between Viterbi and Estimation steps until you converge (or take a fixed number of steps)

* Now use the Viterbi algorithm to compute the best tags for the test set.  Compare the results to those you obtained from Part 0 (when you knew the tags).Is it better?  Worse?

### Solution 1

In [ ]:
# Upload training data file
train_file = files.upload()

In [ ]:
def initialize_viterbi():
  tag_given_tag_counts=dict()
  word_given_tag_counts=dict()

  word_sequence = []
  with open("pos_train.txt", "r") as infile: 
    for line in infile:
        #
        # first tag is the start symbol
        lasttag="<s>"
        #
        # split line into word/tag pairs
        #
        for wordtag in str(line).rstrip().split(" "):
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2 
            parts=wordtag.split("/")
            tag=parts.pop()
            word="/".join(parts)

            word_sequence.append(word)

            #If statement
            if tag not in word_given_tag_counts:
                word_given_tag_counts[tag]=Counter()
            if lasttag not in tag_given_tag_counts:
                tag_given_tag_counts[lasttag]=Counter()
            word_given_tag_counts[tag][word]=1
            tag_given_tag_counts[lasttag][tag]=1
            lasttag=tag

  sentences=[]
  with open("pos_train.txt", "r") as infile:    
      for line in infile:   # line is one of sentence in this file
                        
          # first tag is the start symbol
          lasttag = "<s>" 
          # split line into word/tag pairs
          sentence={}
          for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
              if wordtag == "":
                  continue
              # note that you might have escaped slashes
              # 1\/2/CD means "1/2" "CD"
              # keep 1/2 as 1\/2
              parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']
              #print(parts[1], parts[0])            
              sentence[parts[0]]=parts[1]
              #tag = parts.pop()   # only shows tag of this word  ex) NNS           
              #word = "/".join(parts)   # only shows a word   ex) years            
              #sentence[tag]=word        
          sentences.append(sentence)
  # infile.close()
  ## Obtain state sequence
  tag_sequence = []
  for tag in word_given_tag_counts.keys():
    if tag not in tag_sequence:
      tag_sequence.append(tag)

  tag_given_tag = pd.DataFrame(tag_given_tag_counts)
  word_given_tag = pd.DataFrame(word_given_tag_counts)
  tag_given_tag = tag_given_tag.fillna(0)
  tag_given_tag = tag_given_tag.transpose()

  word_given_tag = word_given_tag.fillna(0)
  word_given_tag = word_given_tag.transpose()

  tagData,wordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)
  probtag, probword=getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts)
  logprobtag, logprobword=logprob(probtag, probword)

  viterbi_tag_seq = []
  for sent in sentences:
    trainVit=viterbi(sent,logprobtag, logprobword)
    viterbi_tag_seq.append(trainVit)
    print(sent)
    print(viterbi_tag_seq)
    
  return(sentences, viterbi_tag_seq)

In [ ]:
def create_new_file(sentences, viterbi_tag_seq):
  #Create file with new tags
  with open("pos_train.txt", "w") as infile: 
    i = 0
    for sent in sentences:
      sent_list = list(sent.keys())
      for j in range (0,len(sent_list)):
        infile.write(sent_list[j] + "/" + viterbi_tag_seq[i][j] + " ")
      i += 1
      infile.write("\n")
  # updated_file.close()

In [ ]:
def train_viterbi():
  #Use file above to create new probability distributions
  updated_tag_given_tag_counts = dict()  # count of each tag
  updated_word_given_tag_counts = dict() # count 

  with open("pos_train.txt", "r") as infile:    
      for line in infile:   # line is one of sentence in this file 
                        
          # first tag is the start symbol
          lasttag = "<s>" 
          # split line into word/tag pairs
          for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
              if wordtag == "":
                  continue
              # note that you might have escaped slashes
              # 1\/2/CD means "1/2" "CD"
              # keep 1/2 as 1\/2
              parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
              tag = parts.pop()   # only shows tag of this word  ex) NNS           
              word = "/".join(parts)   # only shows a word   ex) years

              # update counters
              if tag not in updated_word_given_tag_counts:          # add new tag in word dictionary        
                  updated_word_given_tag_counts[tag] = Counter()                
              if lasttag not in updated_tag_given_tag_counts:       # add new tag in tag dictionary
                  updated_tag_given_tag_counts[lasttag] = Counter()
              updated_word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
              updated_tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
              lasttag = tag   

  ## Obtain state sequence
  tag_sequence = []
  for tag in word_given_tag_counts.keys():
    if tag not in tag_sequence:
      tag_sequence.append(tag)

  tag_given_tag = pd.DataFrame(tag_given_tag_counts)
  word_given_tag = pd.DataFrame(word_given_tag_counts)
  tag_given_tag = tag_given_tag.fillna(0)
  tag_given_tag = tag_given_tag.transpose()

  word_given_tag = word_given_tag.fillna(0)
  word_given_tag = word_given_tag.transpose()


  updatedTagData, updatedWordData=getDataframe(updated_tag_given_tag_counts, updated_word_given_tag_counts)
  updatedProbtag, updatedProbword=getProb(updatedTagData, updatedWordData,updated_tag_given_tag_counts, updated_word_given_tag_counts)
  updatedLogprobtag, updatedLogprobword=logprob(updatedProbtag, updatedProbword)

  for sent in sentences:
      trainVit=viterbi(sent,logprobtag, logprobword)
      viterbi_tag_seq.append(trainVit)
      print(sent)
      print(viterbi_tag_seq)

  return viterbi_tag_seq

In [ ]:
def test_viterbi():

  updated_tag_given_tag_counts = dict()  # count of each tag
  updated_word_given_tag_counts = dict() # count 

  with open("pos_train.txt", "r") as infile:    
      for line in infile:   # line is one of sentence in this file 
                        
          # first tag is the start symbol
          lasttag = "<s>" 
          # split line into word/tag pairs
          for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
              if wordtag == "":
                  continue
              # note that you might have escaped slashes
              # 1\/2/CD means "1/2" "CD"
              # keep 1/2 as 1\/2
              parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
              tag = parts.pop()   # only shows tag of this word  ex) NNS           
              word = "/".join(parts)   # only shows a word   ex) years

              # update counters
              if tag not in updated_word_given_tag_counts:          # add new tag in word dictionary        
                  updated_word_given_tag_counts[tag] = Counter()                
              if lasttag not in updated_tag_given_tag_counts:       # add new tag in tag dictionary
                  updated_tag_given_tag_counts[lasttag] = Counter()
              updated_word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
              updated_tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
              lasttag = tag   

  ## Obtain state sequence
  tag_sequence = []
  for tag in word_given_tag_counts.keys():
    if tag not in tag_sequence:
      tag_sequence.append(tag)

  tag_given_tag = pd.DataFrame(tag_given_tag_counts)
  word_given_tag = pd.DataFrame(word_given_tag_counts)
  tag_given_tag = tag_given_tag.fillna(0)
  tag_given_tag = tag_given_tag.transpose()

  word_given_tag = word_given_tag.fillna(0)
  word_given_tag = word_given_tag.transpose()


  updatedTagData, updatedWordData=getDataframe(updated_tag_given_tag_counts, updated_word_given_tag_counts)
  updatedProbtag, updatedProbword=getProb(updatedTagData, updatedWordData,updated_tag_given_tag_counts, updated_word_given_tag_counts)
  updatedLogprobtag, updatedLogprobword=logprob(updatedProbtag, updatedProbword)

  test_sentences=[]
  with open("pos_test.txt", "r") as infile:    
      for line in infile:   # line is one of sentence in this file
                        
          # first tag is the start symbol
          lasttag = "<s>" 
          # split line into word/tag pairs
          test_sentence={}
          for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
              if wordtag == "":
                  continue
              # note that you might have escaped slashes
              # 1\/2/CD means "1/2" "CD"
              # keep 1/2 as 1\/2
              parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']
              #print(parts[1], parts[0])            
              test_sentence[parts[0]]=parts[1]
              #tag = parts.pop()   # only shows tag of this word  ex) NNS           
              #word = "/".join(parts)   # only shows a word   ex) years            
              #sentence[tag]=word        
          test_sentences.append(test_sentence)

  viterbi_tag_seq = []
  Accuracy=[]
  for sent in test_sentences:
    testVit=viterbi(sent,logprobtag, logprobword)
    viterbi_tag_seq.append(testVit)
    print(sent)
    print(testVit)

    #Accuracy
    b=list(sent.values())
    v = 
    total=len(v)
    correct=0
    for j in range(len(v)):
      if v[j] == b[j]:
        correct+=1

    acc=correct/total
    Accuracy.append(acc)
    print("Accuracy: ", acc)

In [ ]:
#Main method
sentences, viterbi_tag_seq = initialize_viterbi()
for i in range(0, 10): #Stop after 10 iterations
  print("Iteration", i)
  create_new_file(sentences, viterbi_tag_seq)
  viterbi_tag_seq = train_viterbi()

#Then put data set in viterbi
test_viterbi()

### Solution 2

In [ ]:
from collections import Counter
from google.colab import files
import numpy as np
import pandas as pd

In [ ]:
def getDataframe(tag_given_tag_counts, word_given_tag_counts):
  # row is ti-1 and col is ti
  tagData=pd.DataFrame(tag_given_tag_counts)
  tagData=tagData.fillna(0)
  tagData=tagData.transpose()

  wordData=word_given_tag_counts
  wordData=pd.DataFrame(wordData)
  wordData=wordData.fillna(0)
  wordData=wordData.transpose()

  return tagData, wordData

In [ ]:
def getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts):
  # get P(ti|ti-1)
  probtag=tagData.copy()  # to make probability dataframe

  for ti_1 in tag_given_tag_counts.keys():  # get ti-1
    count_ti_1=tagData.loc[ti_1]    # Count of tags in this ti_1 tag
    sumtag=int(np.sum(count_ti_1))   # C(ti-1)
    
    for ti in tag_given_tag_counts.keys():  # get ti 
      if ti != "<s>":      
        tagcount=int(tagData.loc[ti_1][ti])   # this is count(ti-1, ti):   count(ti) in tag ti_1           

        prob_ti_ti_1=tagcount/sumtag   # P(ti | ti-1)
        if prob_ti_ti_1 == 0:
          probtag.loc[ti_1][ti] = 0.000001   # if 0 probability: count 0 words, make it as very small probability
        else:
          probtag.loc[ti_1][ti]=prob_ti_ti_1
    

  # get P(wi|ti)
  probword=wordData.copy()
  words=wordData.columns.tolist()  # all words

  for ti in word_given_tag_counts.keys():  # get ti (tag)
    count_ti=tagData.loc[ti]
    sumtag=int(np.sum(count_ti))

    for wi in word_given_tag_counts[ti]:  # get wi (word)         
      wordcount=int(wordData.loc[ti][wi])

      prob_wi_ti=wordcount/sumtag   # P(wi | ti)
      probword.loc[ti][wi]=prob_wi_ti

  probword+=0.000001  # to erase zeros in probability distribution,  add very small probability

  return probtag, probword

In [ ]:
def logprob(probtag, probword):
  logprobtag=probtag.copy()  # to get log probability distribution of tag_tag pair

  logprobtag=np.log(logprobtag)
    
  # get log P(wi|ti)
  logprobword=probword.copy()   # to get log probability distribution of word_tag pair
  logprobword=np.log(logprobword)

  return logprobtag, logprobword

In [ ]:
def viterbi(sentence, logprobtag, logprobword):  # sequence is a sentence
  #print("hidden: ", list(sentence.keys()))
  #print("observation: ", list(sentence.values()))

  
  tags=logprobtag.columns    # tags of the transition Matrix except <s>
  words=list(sentence.keys())   # words in this sentence
  
 
  # Initialization
  # M = P(tagi) * P(wordi|tagi) as viterbi probability : prior * emission
  M={}  # at start, P(tagi) = P(tagi| "<s>" )
  for tag in tags:     # tags of this sentenc words
    M[tag]=logprobtag.loc["<s>"][tag] + logprobword.loc[tag][words[0]]  # word[0] is the first word of this sentence

  pre=[]

  # Vt(j) = max( Vt-1(i) * P(tagj|tagi) * P(wordj|tagj) )
  
  for t in range(1, len(sentence)):   # Current state: t        
 
    probVit={}   # viterbi probability at t State:  P(Viterbi at j State) * P(tagj|tagi) * P(wordj|tagj)
                # this will store previous viterbi probabilities of current State

    preState={}  # this will store previous tags of current State

    for j in tags:     # j  Current State
      max_prob=-10000  # just set very small number  prob=0 -> logprob ~ very small number
      max_state=None

      for i in tags:   # i   previous State 
                                            #      previous Viterbi
        prob=M[i] + logprobtag.loc[j][i]  #  P(Viterbi at i State) * P(tagj|tagi)
        #print(prev[tagii], logprobtag.loc[tagi][tagj], '   ', prob) 

        #print(prob, max_prob) 
              
        if prob > max_prob:  # change max probability     
          max_prob=prob      # get max of P(Viterbi at i State) * P(tagj|tagi)
          max_state=i  # tag which has a maximum probability 

      if words[t] not in logprobword.columns:  # if this word is not in the dictionary, set prob as very small number
        probV=max_prob - 100
      else:
        probV = max_prob + logprobword.loc[j][words[t]]  # viterbi probability of this State:   P(Viterbi at i State) * P(tagj|tagi) * P(wordi|tagi)

      probVit[j] = probV     # store viterbi probability of this State into this dictionary
      preState[j]=max_state  # { 'DT' : 'IN' } means at tag 'DT' of this sentence, 'IN' shows maximum probability  
    M=probVit
    pre.append(preState)

  #print(M)
  # find hidden states with back tracking
  max_state=None     
  max_prob=-10000
  for tag in tags:    
    if M[tag] > max_prob:
      max_prob=M[tag]
      max_state=tag

  result = [max_state]   # last word's tag in this sentence
  #print(result)

  for t in range(len(sentence)-1, 0, -1):  # from last -> first word 
    max_state=pre[t-1][max_state]

    result.insert(0, max_state)

  #print("Predicted tags: ", result)

  return result

In [ ]:
def read_file(file):
  tag_given_tag_counts = dict()  # count of each tag
  word_given_tag_counts = dict() # count 
  with open(file, "r") as infile:    
    for line in infile:   # line is one of sentence in this file 
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']         
            tag = parts.pop()   # only shows tag of this word  ex) NNS           
            word = "/".join(parts)   # only shows a word   ex) years

            # update counters
            if tag not in word_given_tag_counts:          # add new tag in word dictionary        
                word_given_tag_counts[tag] = Counter()                
            if lasttag not in tag_given_tag_counts:       # add new tag in tag dictionary
                tag_given_tag_counts[lasttag] = Counter()
            word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
            tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
            lasttag = tag  
            
  return word_given_tag_counts, tag_given_tag_counts

In [ ]:
def process_sent_dict(sentence):
  complete_sentence = list()
  pair = {'.': '.'}
  for key, value in sentence.items():
    complete_sentence.append((key, value))
  if list(sentence.keys())[-1] is not '.':
    complete_sentence.append((".", "."))
  complete_sentence = dict(complete_sentence)
  return complete_sentence

In [ ]:
def read_sentences(file):
  sentences = []
  with open(file, "r") as infile:    
    for line in infile:
      # first tag is the start symbol
      lasttag = "<s>" 
      # split line into word/tag pairs
      sentence = {}
      complete_sent = list()
      for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
        if wordtag == "":
          continue
        # note that you might have escaped slashes
        # 1\/2/CD means "1/2" "CD"
        # keep 1/2 as 1\/2
        parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
        sentence[parts[0]]=parts[1]
        complete_sent = process_sent_dict(sentence)     
      sentences.append(complete_sent)

  return sentences

In [ ]:
def write_to_file(file, sentences):
  with open(file, "w") as outputfile:
    for sent in sentences: 
      predicted=viterbi(sent,updatedlogprobtag, updatedlogprobword)
      reform_sent = ""  
      word_sequence = []
      for word in sent.keys():
        word_sequence.append(word)
      for i in range(len(predicted)):
        reform_sent += str(word_sequence[i]) + "/" + str(predicted[i]) + " "
      reform_sent = reform_sent + '\n'
      outputfile.write(reform_sent)
  print("Finish writing!")
  outputfile.close()

In [ ]:
def dataframe_to_array(DataFrame1, DataFrame2):
  logprobtag = DataFrame1.to_numpy()
  logprobtag = np.array(logprobtag)
  logprobword = DataFrame2.to_numpy()
  logprobword = np.array(logprobword)
  return logprobtag, logprobword

In [ ]:
# Upload training data file
train_file = files.upload()

Saving pos_train_test.txt to pos_train_test (2).txt


In [ ]:
filename = "pos_train_test.txt"

In [ ]:
word_given_tag_counts, tag_given_tag_counts = read_file(filename)
sentences = read_sentences(filename)
print(sentences)

[{'Pierre': 'NNP', 'Vinken': 'NNP', ',': ',', '61': 'IN', 'years': 'NNS', 'old': 'IN', 'will': 'DT', 'join': 'VB', 'the': 'DT', 'board': 'NN', 'as': 'IN', 'a': 'DT', 'nonexecutive': 'IN', 'director': 'NN', 'Nov.': 'NNP', '29': 'CD', '.': '.', 'Mr.': 'NNP', 'is': 'VBZ', 'chairman': 'NN', 'of': 'IN', 'Elsevier': 'NNP', 'N.V.': 'NNP', 'Dutch': 'NNP', 'publishing': 'IN', 'group': 'NN'}, {'Rudolph': 'NNP', 'Agnew': 'NNP', ',': ',', '55': 'IN', 'years': 'NNS', 'old': 'JJ', 'and': 'CC', 'former': 'JJ', 'chairman': 'NN', 'of': 'IN', 'Consolidated': 'NNP', 'Gold': 'NNP', 'Fields': 'NNP', 'PLC': 'NNP', 'was': 'VBD', 'named': 'NN', 'a': 'DT', 'nonexecutive': 'IN', 'director': 'NN', 'this': 'DT', 'British': 'IN', 'industrial': 'JJ', 'conglomerate': 'NN', '.': '.'}, {'A': 'DT', 'form': 'NN', 'of': 'IN', 'asbestos': 'NN', 'once': 'IN', 'used': 'NN', 'to': 'TO', 'make': 'VB', 'Kent': 'DT', 'cigarette': 'IN', 'filters': 'DT', 'has': 'VBZ', 'caused': 'NN', 'a': 'DT', 'high': 'IN', 'percentage': 'NN', '

In [ ]:
#Initialization
word_given_tag_counts, tag_given_tag_counts = read_file(filename)
sentences = read_sentences(filename)
updatedtagData,updatedwordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)
updatedprobtag, updatedprobword=getProb(updatedtagData, updatedwordData,tag_given_tag_counts, word_given_tag_counts)
updatedlogprobtag, updatedlogprobword=logprob(updatedprobtag, updatedprobword)
logprobtag, logprobword = dataframe_to_array(updatedlogprobtag, updatedlogprobword)
probability_array = np.array([logprobtag, logprobword], dtype=object)

In [ ]:
# Training
converged = False
epoch = 0
while not converged:
  old_probability = probability_array
  write_to_file(filename, sentences)
  epoch += 1
  for i in range(100):
    word_given_tag_counts, tag_given_tag_counts = read_file(filename)
    sentences = read_sentences(filename)
    updatedtagData,updatedwordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)
    updatedprobtag, updatedprobword=getProb(updatedtagData, updatedwordData,tag_given_tag_counts, word_given_tag_counts)
    updatedlogprobtag, updatedlogprobword=logprob(updatedprobtag, updatedprobword)
    logprobtag, logprobword = dataframe_to_array(updatedlogprobtag, updatedlogprobword)
    probability_array = np.array([logprobtag, logprobword], dtype=object)
  diff = np.sum(np.abs(probability_array-old_probability))
  converged=diff<0.00000001
  if epoch % 10 == 0:
    print("Epoch: %d " % epoch)
    print("Average Error: %f" % diff)
  if epoch > 500: break

print("Converged at epoch "+str(epoch))

In [ ]:
# Upload testing data file
testing_file = files.upload()

Saving pos_test.txt to pos_test (2).txt


In [ ]:
filename_1 = "pos_test.txt"

In [ ]:
word_given_tag_counts, tag_given_tag_counts = read_file(filename_1)
sentences = read_sentences(filename_1)
print(sentences)

[{'No': 'RB', ',': ',', 'it': 'PRP', 'was': 'VBD', "n't": 'RB', 'Black': 'NNP', 'Monday': 'NNP', '.': '.'}, {'But': 'CC', 'while': 'IN', 'the': 'DT', 'New': 'NNP', 'York': 'NNP', 'Stock': 'NNP', 'Exchange': 'NNP', 'did': 'VBD', "n't": 'RB', 'fall': 'VB', 'apart': 'RB', 'Friday': 'NNP', 'as': 'IN', 'Dow': 'NNP', 'Jones': 'NNP', 'Industrial': 'NNP', 'Average': 'NNP', 'plunged': 'VBD', '190.58': 'CD', 'points': 'NNS', '--': ':', 'most': 'JJS', 'of': 'IN', 'it': 'PRP', 'in': 'IN', 'final': 'JJ', 'hour': 'NN', 'barely': 'RB', 'managed': 'VBD', 'to': 'TO', 'stay': 'VB', 'this': 'DT', 'side': 'NN', 'chaos': 'NN', '.': '.'}, {'Some': 'DT', '``': '``', 'circuit': 'NN', 'breakers': 'NNS', "''": "''", 'installed': 'VBN', 'after': 'IN', 'the': 'DT', 'October': 'NNP', '1987': 'CD', 'crash': 'NN', 'failed': 'VBD', 'their': 'PRP$', 'first': 'JJ', 'test': 'NN', ',': ',', 'traders': 'NNS', 'say': 'VBP', 'unable': 'JJ', 'to': 'TO', 'cool': 'VB', 'selling': 'NN', 'panic': 'NN', 'in': 'IN', 'both': 'DT', 

In [ ]:
Accuracy=[]
for i in sentences:
  for j in i.keys():
    print(j, end=" ")
  print()

  v=viterbi(i,updatedlogprobtag, updatedlogprobword)

  #print("hidden State: ", list(i.keys()))
  #print("Predicted State :", v)

  b=list(i.values())

  total=len(v)
  correct=0
  for j in range(len(v)):
    if v[j] == b[j]:
      correct+=1

  acc=correct/total
  Accuracy.append(acc)
  print("Accuracy: ", acc)

# Extension 3

**Extension 3 (3 points):  What happens if you make the words case-insensitive (both in training and test) using the model in Part 0?**

In [ ]:
from google.colab import files
data=files.upload()    # upload pos_train.txt into Colab 
test=files.upload()    # upload pos_test.txt into Colab  

Saving pos_train.txt to pos_train (2).txt


Saving pos_test.txt to pos_test (2).txt


In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

# this is an example of how to parse the POS tag file and get counts
# needed for a bigram tagger

tag_given_tag_counts = dict()  # count of each tag
word_given_tag_counts = dict()
dict((k.lower(), v) for k,v in {'My Key':'My Value'}.items())

with open("pos_train.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file 
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS 
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
            tag = parts.pop()   # only shows tag of this word  ex) NNS           
            word = "/".join(parts).lower()   # only shows a word   ex) years

            # update counters
            if tag not in word_given_tag_counts:          # add new tag in word dictionary        
                word_given_tag_counts[tag] = Counter()                
            if lasttag not in tag_given_tag_counts:       # add new tag in tag dictionary
                tag_given_tag_counts[lasttag] = Counter()
            word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
            tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
            lasttag = tag          
       


# examples
print (word_given_tag_counts)
print("count[NN][VB] = " + str(tag_given_tag_counts["NN"]["VB"]))   # means  after tag 'NN' , 'VB' shows 229 times.   t1 is 'NN' t2 is 'VB' and  result is Count(t1, t2)
print("count[NN][dog] = " + str(word_given_tag_counts["NN"]["dog"]))  # means  after tag 'NN', 'dog' shows 12 times.  t1 is 'NN' w1 is 'dog' and  result is Count(t1, w1)
print("count[NN][down] = " + str(word_given_tag_counts["NN"]["down"]))  # means  after tag 'NN', 'dog' shows 12 times.  t1 is 'NN' w1 is 'dog' and  result is Count(t1, w1)


{'NNP': Counter({'mr.': 5081, 'u.s.': 1931, 'new': 1517, 'corp.': 1471, 'inc.': 1307, 'york': 1187, 'co.': 1090, 'exchange': 600, 'friday': 587, 'japan': 507, 'house': 474, 'bank': 461, 'national': 441, 'american': 439, 'bush': 432, 'oct.': 421, 'stock': 418, 'treasury': 411, 'congress': 404, 'federal': 395, 'wall': 381, 'street': 379, 'ual': 369, 'california': 368, 'san': 366, 'board': 363, 'dow': 362, 'september': 360, 'international': 344, 'monday': 341, 'london': 331, 'group': 328, 'jones': 326, 'john': 320, 'nov.': 314, 'chicago': 312, 'august': 305, 'department': 304, 'president': 303, 'ms.': 285, 'inc': 284, 'francisco': 273, 'big': 268, 'first': 267, 'senate': 267, 'robert': 257, 'october': 256, 'washington': 255, 'texas': 249, 'general': 247, 'tokyo': 245, 'ford': 245, 'tuesday': 243, 'ltd.': 232, 'james': 229, 'west': 229, 'america': 229, 'association': 225, 'fed': 225, 'united': 222, 'july': 220, 'co': 220, 'europe': 220, 'warner': 219, 'june': 215, 'corp': 214, 'union': 210

In [ ]:
# Make Dataframe of each tag_tag and tag_word

def getDataframe(tag_given_tag_counts, word_given_tag_counts):
  # row is ti-1 and col is ti
  tagData=pd.DataFrame(tag_given_tag_counts)
  tagData=tagData.fillna(0)
  tagData=tagData.transpose()
  #print(tagData)

  wordData=word_given_tag_counts
  wordData=pd.DataFrame(wordData)
  wordData=wordData.fillna(0)
  wordData=wordData.transpose()
  #print(wordData)


  #print(list(wordData.columns))

  #print(tagData.loc["NNP"].tolist())
  #print(wordData.loc["NNP"].tolist())
  #print(word_given_tag_counts["NNP"]["rarsasrtat"])

  return tagData, wordData

#print(wordData.columns.tolist())
#print(str(word_given_tag_counts["NNP"]))

#tagData, wordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)

print(tagData)
print(wordData)

              NNP      DT       IN      ``  ...  VBD|VBP  VBP|VBD  JJ|JJR  IN|JJ
<s>        6355.0  4649.0   3058.0  1194.0  ...      0.0      0.0     0.0    0.0
NNP       43427.0   273.0   4688.0   122.0  ...      0.0      0.0     0.0    0.0
,          7746.0  8120.0   5131.0   792.0  ...      0.0      0.0     0.0    0.0
CD          613.0  1309.0   4039.0    33.0  ...      0.0      0.0     0.0    0.0
NNS         213.0  1237.0  17437.0   171.0  ...      0.0      0.0     0.0    0.0
...           ...     ...      ...     ...  ...      ...      ...     ...    ...
NN|POS        0.0     0.0      0.0     0.0  ...      0.0      0.0     0.0    0.0
IN|JJ         0.0     0.0      0.0     0.0  ...      0.0      0.0     0.0    0.0
JJ|VBN        0.0     0.0      0.0     0.0  ...      0.0      0.0     0.0    0.0
NNS|NNPS      0.0     0.0      0.0     0.0  ...      0.0      0.0     0.0    0.0
VBD|VBN       0.0     0.0      0.0     0.0  ...      0.0      0.0     0.0    0.0

[80 rows x 79 columns]
    

In [ ]:
# we have to calcualte P(ti | ti_1) = count(ti-1, ti)/count(ti-1)

def getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts):
  # get P(ti|ti-1)
  probtag=tagData.copy()  # to make probability dataframe

  for ti_1 in tag_given_tag_counts.keys():  # get ti-1
    count_ti_1=tagData.loc[ti_1]    # Count of tags in this ti_1 tag
    sumtag=int(np.sum(count_ti_1))   # C(ti-1)
    
    for ti in tag_given_tag_counts.keys():  # get ti 
      if ti != "<s>":      
        tagcount=int(tagData.loc[ti_1][ti])   # this is count(ti-1, ti):   count(ti) in tag ti_1           

        prob_ti_ti_1=tagcount/sumtag   # P(ti | ti-1)
        if prob_ti_ti_1 == 0:
          probtag.loc[ti_1][ti] = 0.000001   # if 0 probability: count 0 words, make it as very small probability
        else:
          probtag.loc[ti_1][ti]=prob_ti_ti_1

  #print(probtag)   # each element is P(ti|ti-1)  ex) P(NNP|<s>) = 0.282620
    

  # get P(wi|ti)
  probword=wordData.copy()
  words=wordData.columns.tolist()  # all words

  for ti in word_given_tag_counts.keys():  # get ti (tag)
    count_ti=tagData.loc[ti]
    sumtag=int(np.sum(count_ti))

    for wi in word_given_tag_counts[ti]:  # get wi (word)         
      wordcount=int(wordData.loc[ti][wi])

      prob_wi_ti=wordcount/sumtag   # P(wi | ti)
      probword.loc[ti][wi]=prob_wi_ti

  probword+=0.000001  # to erase zeros in probability distribution,  add very small probability

  #print(probword)  # each element is P(wi|ti)  ex) P(Pierre|NNP) = 0.000052
  #print(probword.loc["NNP"]["Pierre"])
  #print(len(word_given_tag_counts["NNP"]))

  return probtag, probword


#probtag, probword=getProb(tagData, wordData)
#print(probtag)
#print(probword)

In [ ]:
# get log P(ti|ti-1)
def logprob(probtag, probword):
  logprobtag=probtag.copy()  # to get log probability distribution of tag_tag pair

  logprobtag=np.log(logprobtag)

  #print(logprobtag)   # each element is log P(ti|ti-1)  ex) log P(NNP|<s>) = -1.262651
    

  # get log P(wi|ti)
  logprobword=probword.copy()   # to get log probability distribution of word_tag pair
  logprobword=np.log(logprobword)

  #print(logprobword)  # each element is log P(wi|ti)  ex) log P(Pierre|NNP) = -9.838250
  return logprobtag, logprobword

#logprobtag, logprobword=logprob(probtag, probword)
#print(logprobtag)
#print(logprobword)

In [ ]:
# just check the Viterbi Algorithm, take one sentence in the dataset

sentences=[]
with open("pos_train.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        sentence={}
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")    # foramt as ['word', 'tag']  ex) ['years', 'NNS']
            #print(parts[1], parts[0])            
            sentence[parts[0].lower()]=parts[1]
            #tag = parts.pop()   # only shows tag of this word  ex) NNS           
            #word = "/".join(parts)   # only shows a word   ex) years            
            #sentence[tag]=word        
        sentences.append(sentence)
        
        
# one sentence        
sent=sentences[10]
print(sent)

{'the': 'DT', 'percentage': 'NN', 'of': 'IN', 'lung': 'NN', 'cancer': 'NN', 'deaths': 'NNS', 'among': 'IN', 'workers': 'NNS', 'at': 'IN', 'west': 'NNP', 'groton': 'NNP', ',': ',', 'mass.': 'NNP', 'paper': 'NN', 'factory': 'NN', 'appears': 'VBZ', 'to': 'TO', 'be': 'VB', 'highest': 'JJS', 'for': 'IN', 'any': 'DT', 'asbestos': 'NN', 'studied': 'VBN', 'in': 'IN', 'western': 'JJ', 'industrialized': 'VBN', 'countries': 'NNS', 'he': 'PRP', 'said': 'VBD', '.': '.', 'plant': 'NN', 'which': 'WDT', 'is': 'VBZ', 'owned': 'VBN', 'by': 'IN', 'hollingsworth': 'NNP', '&': 'CC', 'vose': 'NNP', 'co.': 'NNP', 'was': 'VBD', 'under': 'IN', 'contract': 'NN', 'with': 'IN', 'lorillard': 'NN', 'make': 'VB', 'cigarette': 'NN', 'filters': 'NNS'}


In [ ]:
# Viterbi Algorithm

#tags=list(sent.keys())
#words=list(sent.values())

tagData,wordData=getDataframe(tag_given_tag_counts, word_given_tag_counts)
probtag, probword=getProb(tagData, wordData,tag_given_tag_counts, word_given_tag_counts)
logprobtag, logprobword=logprob(probtag, probword)


## use log probability
def viterbi(sentence, logprobtag, logprobword):  # sequence is a sentence
  #print("hidden: ", list(sentence.keys()))
  #print("observation: ", list(sentence.values()))

  
  tags=logprobtag.columns    # tags of the transition Matrix except <s>
  words=list(sentence.keys())   # words in this sentence
  print(words)
  # Initialization
  # M = P(tagi) * P(wordi|tagi) as viterbi probability : prior * emission
  M={}  # at start, P(tagi) = P(tagi| "<s>" )
  for tag in tags:     # tags of this sentenc words
    M[tag]=logprobtag.loc["<s>"][tag] + logprobword.loc[tag][words[0]]  # word[0] is the first word of this sentence

  pre=[]

  # Vt(j) = max( Vt-1(i) * P(tagj|tagi) * P(wordj|tagj) )
  
  for t in range(1, len(sentence)):   # Current state: t        
 
    probVit={}   # viterbi probability at t State:  P(Viterbi at j State) * P(tagj|tagi) * P(wordj|tagj)
                # this will store previous viterbi probabilities of current State

    preState={}  # this will store previous tags of current State

    for j in tags:     # j  Current State
      max_prob=-10000  # just set very small number  prob=0 -> logprob ~ very small number
      max_state=None

      for i in tags:   # i   previous State 
                                            #      previous Viterbi
        prob=M[i] + logprobtag.loc[j][i]  #  P(Viterbi at i State) * P(tagj|tagi)
        #print(prev[tagii], logprobtag.loc[tagi][tagj], '   ', prob) 

        #print(prob, max_prob) 
              
        if prob > max_prob:  # change max probability     
          max_prob=prob      # get max of P(Viterbi at i State) * P(tagj|tagi)
          max_state=i  # tag which has a maximum probability 

      if words[t] not in logprobword.columns:  # if this word is not in the dictionary, set prob as very small number
        probV=max_prob - 100
      else:
        probV = max_prob + logprobword.loc[j][words[t]]  # viterbi probability of this State:   P(Viterbi at i State) * P(tagj|tagi) * P(wordi|tagi)

      probVit[j] = probV     # store viterbi probability of this State into this dictionary
      preState[j]=max_state  # { 'DT' : 'IN' } means at tag 'DT' of this sentence, 'IN' shows maximum probability  
    M=probVit
    pre.append(preState)

  #print(M)
  # find hidden states with back tracking
  max_state=None     
  max_prob=-10000
  for tag in tags:    
    if M[tag] > max_prob:
      max_prob=M[tag]
      max_state=tag

  result = [max_state]   # last word's tag in this sentence
  #print(result)

  for t in range(len(sentence)-1, 0, -1):  # from last -> first word 
    max_state=pre[t-1][max_state]

    result.insert(0, max_state)

  #print("Predicted tags: ", result)

  return result
    

trainVit=viterbi(sent,logprobtag, logprobword)
print(trainVit)
  
  



['the', 'percentage', 'of', 'lung', 'cancer', 'deaths', 'among', 'workers', 'at', 'west', 'groton', ',', 'mass.', 'paper', 'factory', 'appears', 'to', 'be', 'highest', 'for', 'any', 'asbestos', 'studied', 'in', 'western', 'industrialized', 'countries', 'he', 'said', '.', 'plant', 'which', 'is', 'owned', 'by', 'hollingsworth', '&', 'vose', 'co.', 'was', 'under', 'contract', 'with', 'lorillard', 'make', 'cigarette', 'filters']
['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'DT', 'TO', 'VB', 'JJS', 'IN', 'DT', 'IN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'TO', 'NNS']


In [ ]:
# Get train data's accuracy.

print("hidden: ", list(sent.values()))
print("Predict:", trainVit)

trainTags=list(sent.values())

total=len(trainVit)
correct=0
for i in range(len(trainVit)):
  if trainVit[i] == trainTags[i]:
    correct+=1

print("Accuracy: ",correct/total)


hidden:  ['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'VBZ', 'TO', 'VB', 'JJS', 'IN', 'DT', 'NN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'NN', 'NNS']
Predict: ['DT', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'NNP', 'NN', 'NN', 'DT', 'TO', 'VB', 'JJS', 'IN', 'DT', 'IN', 'VBN', 'IN', 'JJ', 'VBN', 'NNS', 'PRP', 'VBD', '.', 'NN', 'WDT', 'VBZ', 'VBN', 'IN', 'NNP', 'CC', 'NNP', 'NNP', 'VBD', 'IN', 'NN', 'IN', 'NN', 'VB', 'TO', 'NNS']
Accuracy:  0.9361702127659575


In [ ]:
# Apply my Viterbi algorithm to the test dataset

# Get test.txt's tags and words
test_tag_given_tag_counts = dict()  # count of each tag
test_word_given_tag_counts = dict() # count 

with open("pos_test.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file 
               
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']            
            tag = parts.pop()   # only shows tag of this word  ex) NNS           
            word = "/".join(parts).lower()   # only shows a word   ex) years

            # update counters
            if tag not in test_word_given_tag_counts:          # add new tag in word dictionary        
                test_word_given_tag_counts[tag] = Counter()                
            if lasttag not in test_tag_given_tag_counts:       # add new tag in tag dictionary
                test_tag_given_tag_counts[lasttag] = Counter()
            test_word_given_tag_counts[tag][word] += 1         # in word dictionary, if this tag has this word, count+=1    
            test_tag_given_tag_counts[lasttag][tag] += 1       # in tag dictionary, if thi tag has this tag, count+=1     
            lasttag = tag          

testtagData,testwordData=getDataframe(test_tag_given_tag_counts, test_word_given_tag_counts)
testprobtag, testprobword=getProb(testtagData, testwordData,test_tag_given_tag_counts, test_word_given_tag_counts)
testlogprobtag, testlogprobword=logprob(testprobtag, testprobword)

In [ ]:
test_sentences=[]
with open("pos_test.txt", "r") as infile:    
    for line in infile:   # line is one of sentence in this file
                       
        # first tag is the start symbol
        lasttag = "<s>" 
        # split line into word/tag pairs
        test_sentence={}
        for wordtag in line.rstrip().split(" "):    # each word and tag in this sentence. foramt as word/tag  ex) years/NNS           
            if wordtag == "":
                continue
            # note that you might have escaped slashes
            # 1\/2/CD means "1/2" "CD"
            # keep 1/2 as 1\/2
            parts = wordtag.split("/")     # foramt as ['word', 'tag']  ex) ['years', 'NNS']
            #print(parts[1], parts[0])            
            test_sentence[parts[0].lower()]=parts[1]
            #tag = parts.pop()   # only shows tag of this word  ex) NNS           
            #word = "/".join(parts)   # only shows a word   ex) years            
            #sentence[tag]=word        
        test_sentences.append(test_sentence)
        
        
# one sentence        
test_sent=test_sentences[6]
print(test_sent)

{'countered': 'VBD', 'james': 'NNP', 'maguire': 'NNP', ',': ',', 'chairman': 'NN', 'of': 'IN', 'specialists': 'NNS', 'henderson': 'NNP', 'brothers': 'NNP', 'inc.': 'NNP', ':': ':', '``': '``', 'it': 'PRP', 'is': 'VBZ', 'easy': 'JJ', 'to': 'TO', 'say': 'VB', 'the': 'DT', 'specialist': 'NN', "n't": 'RB', 'doing': 'VBG', 'his': 'PRP$', 'job': 'NN', '.': '.', 'when': 'WRB', 'dollar': 'NN', 'in': 'IN', 'a': 'DT', 'free-fall': 'NN', 'even': 'RB', 'central': 'JJ', 'banks': 'NNS', 'ca': 'MD', 'stop': 'VB', 'speculators': 'NNS', 'are': 'VBP', 'calling': 'VBG', 'for': 'IN', 'degree': 'NN', 'liquidity': 'NN', 'that': 'WDT', 'not': 'RB', 'there': 'RB', 'market': 'NN', "''": "''"}


In [ ]:
# test Viterbi algorithm for just one sentence in the test dataset
testVit=viterbi(test_sent, testlogprobtag, testlogprobword)
print(testVit)

['countered', 'james', 'maguire', ',', 'chairman', 'of', 'specialists', 'henderson', 'brothers', 'inc.', ':', '``', 'it', 'is', 'easy', 'to', 'say', 'the', 'specialist', "n't", 'doing', 'his', 'job', '.', 'when', 'dollar', 'in', 'a', 'free-fall', 'even', 'central', 'banks', 'ca', 'stop', 'speculators', 'are', 'calling', 'for', 'degree', 'liquidity', 'that', 'not', 'there', 'market', "''"]
['NNP', 'NNP', 'NNP', ',', 'NNP', 'IN', 'NNS', 'NNP', 'NNP', 'NNP', ':', '``', 'PRP', 'VBZ', 'JJ', 'TO', 'VBP', 'DT', 'IN', 'RB', 'VBG', 'PRP$', 'NN', '.', 'WRB', 'NN', 'IN', 'DT', 'NN', 'JJ', 'JJ', 'NNS', 'JJ', 'VB', 'NNS', 'VBP', 'NNS', 'IN', 'NN', 'NN', 'IN', 'RB', 'RB', 'NN', "''"]


In [ ]:
# Report test Accuracy of each sentence
# It takes too much time to get all result of the entire test dataset
Accuracy=[]
for i in test_sentences:
    
  for j in i.keys():
    print(j, end=" ")
  print()

  v=viterbi(i,testlogprobtag, testlogprobword)

  #print("hidden State: ", list(i.keys()))
  #print("Predicted State :", v)

  b=list(i.values())

  total=len(v)
  correct=0
  for j in range(len(v)):
    if v[j] == b[j]:
      correct+=1

  acc=correct/total
  Accuracy.append(acc)
  print("Accuracy: ", acc)

no , it was n't black monday . 
['no', ',', 'it', 'was', "n't", 'black', 'monday', '.']
Accuracy:  0.875
but while the new york stock exchange did n't fall apart friday as dow jones industrial average plunged 190.58 points -- most of it in final hour barely managed to stay this side chaos . 
['but', 'while', 'the', 'new', 'york', 'stock', 'exchange', 'did', "n't", 'fall', 'apart', 'friday', 'as', 'dow', 'jones', 'industrial', 'average', 'plunged', '190.58', 'points', '--', 'most', 'of', 'it', 'in', 'final', 'hour', 'barely', 'managed', 'to', 'stay', 'this', 'side', 'chaos', '.']
Accuracy:  0.8571428571428571
some `` circuit breakers '' installed after the october 1987 crash failed their first test , traders say unable to cool selling panic in both stocks and futures . 49 stock specialist firms on big board floor -- buyers sellers of last resort who were criticized once again could n't handle pressure 
['some', '``', 'circuit', 'breakers', "''", 'installed', 'after', 'the', 'october', '

In [ ]:
# Viterbi Algorithm's accuracy of test dataset 
print(Accuracy)
mean_Accuracy=np.mean(Accuracy)
print(mean_Accuracy)

[0.875, 0.8571428571428571, 0.9038461538461539, 0.6944444444444444, 0.9166666666666666, 0.9024390243902439, 0.8222222222222222, 0.8904109589041096, 0.9024390243902439, 0.8679245283018868, 0.8478260869565217, 0.851063829787234, 0.8, 0.8636363636363636, 0.8478260869565217, 0.8367346938775511, 0.8115942028985508, 0.7708333333333334, 0.7837837837837838, 0.8222222222222222, 0.95, 0.8947368421052632, 0.8571428571428571, 0.8484848484848485, 0.7941176470588235, 0.8928571428571429, 0.8536585365853658, 0.8863636363636364, 0.7916666666666666, 0.7872340425531915, 0.803030303030303, 0.8253968253968254, 0.8, 0.7884615384615384, 0.8636363636363636, 0.85, 0.875, 1.0, 1.0, 0.7777777777777778, 0.813953488372093, 0.9473684210526315, 0.9, 0.9090909090909091, 0.8409090909090909, 0.7894736842105263, 0.896551724137931, 0.9210526315789473, 0.8372093023255814, 0.8695652173913043, 0.8367346938775511, 0.9166666666666666, 0.85, 0.8409090909090909, 0.8524590163934426, 0.7954545454545454, 0.8666666666666667, 0.9333

In [ ]:
# We tried Viterbi for the rntire test dataset.
# We got: mean of Viterbi accuracy for test dataset as 0.8447598144366402 :  84.47% 
print("Average Viterbi Accuracy: ", mean_Accuracy)

Average Viterbi Accuracy:  0.8447598144366402
